In [1]:
import os

In [2]:
%pwd

'c:\\Users\\karakkir\\OneDrive - Tietoevry\\Kidney-dl\\research'

In [3]:
os.chdir('../')
%pwd

'c:\\Users\\karakkir\\OneDrive - Tietoevry\\Kidney-dl'

In [4]:
os.environ['MLFLOW_TRACKING_URI']='https://dagshub.com/kirankarakalli/Kideny-Disease-Classification-Deep-Learning-Project.dvc'
os.environ['MLFLOW_TRACKING_USERNAME']=' kirankarakalli'
os.environ['MLFLOW_TRACKING_PASSWORD']='87b671153c6abc5700206e41888a56e7d6cb0e49'

In [5]:
import tensorflow as tf 

In [6]:
model=tf.keras.models.load_model('artifacts/training/model.h5')

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_to_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [8]:
from cnnclassifier.constants import *
from cnnclassifier.utils.common import read_yaml, create_directories, save_json

In [9]:
class ConfigurationManager:
    def __init__(self,config_file_path=CONFIG_FILE_PATH,params_file_path=PARAMS_FILE_PATH):
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_eval_config(self)->EvaluationConfig:
        eval_config=EvaluationConfig(
            path_to_model="artifacts/training/model.h5",
            training_data='artifacts/data_ingestion/kidney-ct-scan-image', # type: ignore
            all_params=self.params,
            mlflow_uri='https://dagshub.com/kirankarakalli/Kideny-Disease-Classification-Deep-Learning-Project.dvc',
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )

        return eval_config



In [10]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [15]:
class Evaluation:
    def __init__(self,config:EvaluationConfig):
        self.config=config

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
    def load_model(self,path:Path)->tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_to_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            if tracking_url_type_store != "file":

                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [18]:
try:
    config=ConfigurationManager()
    eval_config=config.get_eval_config()
    eval=Evaluation(eval_config)
    eval.evaluation()
    eval.save_score()
except Exception as e:
    raise e

Found 139 images belonging to 2 classes.
9/9 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.9568 - loss: 1.4812
